In [9]:
# Web scraping stuff
import requests
#!conda install -c anaconda beautifulsoup4 --yes
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [28]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge geocoder --yes
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Pulling Postal Data from Wikipedia
using this tutorial: [bs4](https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722)

## Pulling from wiki

In [11]:
website_url_str = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(website_url_str).text

#soup = BeautifulSoup(website_url,'lxml')
soup = BeautifulSoup(website_url,'lxml')

## Creating raw table

In [12]:
my_table = soup.find('table',{'class':'wikitable sortable'})

table_contents = my_table.find_all('td')
col = 0
post_codes = []
boroughs = []
neighborhoods = []
for tag in table_contents:
    if col == 0:
        post_codes.append(tag.get_text())
    elif col == 1:
        boroughs.append(tag.get_text())
    elif col == 2:
        neighborhoods.append(tag.get_text().strip())
    else:
        print('something is wrong')
        
    col = col + 1
    
    if col >= 3:
        col = 0
        
df = pd.DataFrame()
df['PostalCode'] = post_codes
df['Boroughs'] = boroughs
df['Neighborhoods'] = neighborhoods

## Cleaning up the table

In [13]:
# drop rows with unassigned boroughs
df.Boroughs = df['Boroughs'].replace('Not assigned', np.nan)
df.dropna(subset=['Boroughs'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [14]:
# fill missing neighborhood values with Borough value on same row
df.Neighborhoods = df.Neighborhoods.replace('Not assigned', np.nan)
df.Neighborhoods.fillna(df.Boroughs, inplace=True)

In [15]:
# combining Neighborhood text into comma seperated strings for repeated Postal Codes
df = df.groupby(['PostalCode', 'Boroughs'])['Neighborhoods'].apply(', '.join).reset_index()

# Obtaining Latitudes and Longitudes

In [30]:
#this takes forever, comenting out and importing from csv

'''
lat_lng_coords = None
postal_code = df.PostalCode.values[0]
# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
'''

"\nlat_lng_coords = None\npostal_code = df.PostalCode.values[0]\n# loop until you get the coordinates\nwhile(lat_lng_coords is None):\n  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n  lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]\n"

In [46]:
geo_df = pd.read_csv("Geospatial_Coordinates.csv")
geo_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

In [50]:
df = df.merge(geo_df, left_on='PostalCode', right_on='PostalCode')